### [Skip code and jump to racing](#Race-Time)

In [ ]:
%reload_ext nb_black

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
def _gen_player_data(color, name="", y=0, n_steps=20):
    """Generate a random race time and split it out over time steps for plotting"""
    # Generate random race times
    finish_time = np.random.normal(n_steps * 0.8, 0.3)

    # Find x position for each frame of race
    rate = 1 / finish_time
    step = finish_time / n_steps
    time_steps = np.arange(n_steps + 1)
    x_pos = time_steps * rate

    # Store all plotting info for plotly
    race_df = pd.DataFrame(
        {
            "time": time_steps,
            "x": x_pos,
            "y": y,
            "color": color,
            "name": name,
            "finish_time": finish_time,
        }
    )

    # Add a little jitter to be less boring
    excitement = np.ones_like(x_pos) * 0.01
    excitement[: len(excitement) // 2] *= -1
    np.random.shuffle(excitement)
    race_df["x"] += excitement
    race_df.loc[0, "x"] = 0

    return race_df


def _gen_race_data(players, colors=px.colors.qualitative.T10):
    """'Simulate' a marble race between players"""
    race_dfs = []
    name_colors = zip(players, colors)
    for i, (name, color) in enumerate(name_colors):
        race_df = _gen_player_data(color, name, i * 0.1)
        race_dfs.append(race_df)

    return pd.concat(race_dfs).reset_index(drop=True)


def marble_race(players, seed=None):
    """'Simulate' a marble race"""
    if isinstance(seed, int):
        np.random.seed(seed)
    race_df = _gen_race_data(players)

    return (
        race_df[["color", "name", "finish_time"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )

In [ ]:
def plot_marble_race(players, seed=None):
    """'Simulate' and plot a marble race"""
    if isinstance(seed, int):
        np.random.seed(seed)
    race_df = _gen_race_data(players)

    color_df = race_df[["color", "name"]].drop_duplicates()
    color_discrete_map = {}
    for _, row in color_df.iterrows():
        color_discrete_map[row["name"]] = row["color"]

    fig = px.scatter(
        data_frame=race_df,
        x="x",
        y="y",
        color="name",
        text="name",
        animation_frame="time",
        title="Thinkful Marble Racing Series",
        color_discrete_map=color_discrete_map,
    )

    fig.update_traces(marker={"size": 20})
    fig.update_layout(showlegend=False)

    fig.add_trace(
        go.Scatter(x=[1, 1], y=[-300, 300], mode="lines", line={"color": "black"},)
    )

    fig.update_xaxes(
        {"range": [-0.1, 1.1], "showgrid": False, "zeroline": False, "visible": False,}
    )
    fig.update_yaxes(
        {"range": [-0.1, 1.1], "showgrid": False, "zeroline": False, "visible": False,}
    )

    return fig

# Race Time

In [ ]:
# To get consistent results between
# animation and score board.
# No editing needed.
seed = np.random.randint(0, 1000)

In [ ]:
# Edit this to have the names of who/what is racing.
racers = ["A", "B", "C"]

In [ ]:
# Play out race visually.
plot_marble_race(racers, seed)

In [ ]:
# Show race results.
# Spoiler alert if this cell is run before plot_marble_race().
marble_race(racers, seed).sort_values("finish_time")